<a href="https://colab.research.google.com/github/suzukiiichiro/N-Queens/blob/master/01Bash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bash01

In [ ]:
%%bash 
#
function N-Queen1(){
  local -i min="$1";
  local -i size="$2";
  local -i i=0;
  local -i j=0;
  for((i=0;i<size;i++)){
    pos[$min]="$i";
    ((min==(size-1)))&&{
      echo -n "$((COUNT++)): ";
      for((j=0;j<size;j++)){
        echo -n "${pos[j]} ";
      }
      echo "";
    }||N-Queen1 "$((min+1))" "$size";
  }
}
#
echo "<>1.ブルートフォース（力まかせ探索）N-Queen1()";
N-Queen1 0 4;
#


# Bash02

In [ ]:
%%bash 

#
COUNT=1;
#
function N-Queen2(){
  local -i min=$1;
  local -i size=$2;
  local flag_a="";
  local -i i=0;
  local -i j=0;
  for((i=0;i<size;i++)){
    [ "${flag_a[i]}" != "true" ]&&{
      pos[$min]="$i";
      ((min==(size-1)))&&{
        echo -n "$((COUNT++)): ";
        for((j=0;j<size;j++)){
          echo -n "${pos[j]} ";
        }
        echo "";
      }||{
        flag_a[$i]="true";
        N-Queen2 "$((min+1))" "$size";
        flag_a[$i]="";
      }
    }
  }
}
#
echo "<>2.配置フラグ（制約テスト高速化） N-Queen2()";
N-Queen2 0 4;
#

# Bash03

In [ ]:
%%bash

TOTAL=0;
UNIQUE=0;
typeset -a flag_a="";
typeset -a flag_b="";
typeset -a flag_c="";
#
function N-Queen3_rec(){
    local -i min="$1";
    local -i size="$2";
    local -i i=0;
    for((i=0;i<size;i++)){
        [ "${flag_a[$i]}" != "true" ]&&[ "${flag_b[$min+$i]}" != "true" ]&&[ "${flag_c[$min-$i+$size-1]}" != "true" ]&&{
            pos[$min]=$i;
            ((min==(size-1)))&&{
                ((TOTAL++));
            }||{
                flag_a[$i]="true";
                flag_b[$min+$i]="true";
                flag_c[$min-$i+$size-1]="true";
                N-Queen3_rec "$((min+1))" "$size" ;
                flag_a[$i]="";
                flag_b[$min+$i]="";
                flag_c[$min-$i+$size-1]="";
            }
        }
    }
}
#
function N-Queen3(){
    local -i max=8;
    local -i min=2;
    local -i N="$min";
    local startTime=0;
    local endTime=0;
    local hh=mm=ss=0;
    echo " N:        Total       Unique        hh:mm:ss" ;
    for((N=min;N<=max;N++)){
        TOTAL=0;
        UNIQUE=0;
        startTime=$(date +%s);
        N-Queen3_rec 0 "$N";
        endTime=$(date +%s);
        ss=$((endTime - startTime));
        hh=$((ss/3600));
        ss=$((ss%3600));
        mm=$((ss/60));
        ss=$((ss%60));
        printf "%2d:%13d%13d%10d:%.2d:%.2d" $N $TOTAL $UNIQUE $hh $mm $ss;
        echo "";
    }
}

echo "<>3.バックトラック N-Queen3()";
N-Queen3;

<>3.バックトラック N-Queen3()
 N:        Total       Unique        hh:mm:ss
 2:            0            0         0:00:00
 3:            0            0         0:00:00
 4:            2            0         0:00:00
 5:           10            0         0:00:00
 6:            4            0         0:00:00
 7:           40            0         0:00:00
 8:           92            0         0:00:00


# Bash04

In [ ]:
%%bash

typeset -i TOTAL=0;
typeset -i UNIQUE=0;
typeset -a flag_a="";
typeset -a flag_b="";
typeset -a flag_c="";
typeset -a trial="";
typeset -a board="";
typeset -a scratch="";
#
function intncmp(){
  local -i k;
  local -i rtn=0;
  local -i n=$1;
  for((k=0;k<n;k++)){
    rtn=$((board[k]-trial[k]));
    ((rtn!=0))&&{ break; }
  }
  echo "$rtn";
}
#
function rotate(){
  local -i j;
  local -i k;
  local -i n="$1";
  local -i incr;
  local -i neg="$2";
  if [ "$neg" = "true" ]; then
    k=0;
  else
    k=$((n-1));
  fi
  if [ "$neg" = "true" ]; then
    incr=$((incr+1));
  else
    incr=$((incr-1));
  fi
  for((j=0;j<n;k+=incr)){
    j=$((j+1))
    scratch[$j]=${trial[$k]};
  }
  if [ "$neg" = "true" ]; then
    k=$((n-1));
  else
    k=0;
  fi
  for((j=0;j<n;k-=incr)){
    j=$((j+1))
    trial[${scratch[$j]}]=$k;
  }
}
#
function vMirror(){
  local -i j;
  local -i n="$1";
  for((j=0;j<n;j++)){
    local -i n1=$((n-1));
    trial[$j]=$((n1-trial[j]));
  }
}
#
function symmetryOps(){
  local -i k;
  local -i nEquiv;
  local -i size="$1";
  for((k=0;k<size;k++)){
    trial[$k]=${board[$k]};
  }
  rotate "$size" "false";
  k=$(intncmp "$size");
  ((k>0))&&{
    echo 0;
    return;
  }
  ((k==0))&&{
    nEquiv=1;
  }||{
    rotate "$size" "false";
    k=$(intncmp "$size");
    ((k>0))&&{
      echo 0;
      return;
    }
    ((k==0))&&{
      nEquiv=2;
    }||{
      rotate "$size" "false";
      k=$(intncmp "$size");
      ((k>0))&&{
        echo 0;
        return;
      }
      nEquiv=4;
    }
  }
  for((k=0;k<size;k++)){
    trial[$k]=${board[$k]};
  }
  vMirror "$size";
  k=$(intncmp "$size");
  ((k>0))&&{
    echo 0;
    return;
  }
  ((nEquiv>1))&&{
    rotate "$size" "true";
    k=$(intncmp "$size");
    ((k>0))&&{
      echo 0;
      return;
    }
    ((nEquiv>2))&&{
      rotate "$size" "true";
      k=$(intncmp "$size");
      ((k>0))&&{
        echo 0;
        return ;
      }
      rotate "$size" "true";
      k=$(intncmp "$size");
      ((k>0))&&{
        echo 0;
        return;
      }
    }
  }
  rtn=$((nEquiv * 2));
  echo "$rtn";
  return;
}
#
function N-Queen4_rec(){
  local -i min="$1";
  local -i size="$2";
  local -i i=0;
  for((i=0;i<size;i++)){
    [ "${flag_a[$i]}" != "true" ]&&[ "${flag_b[$min+$i]}" != "true" ]&&[ "${flag_c[$min-$i+$size-1]}" != "true" ]&&{
      board[$min]=$i;
      ((min==(size-1)))&&{
        tst=$(symmetryOps "$size");
        ((tst!=0))&&{
          ((UNIQUE++));
          TOTAL=$((TOTAL+tst));
        }
      }||{
        flag_a[$i]="true";
        flag_b[$min+$i]="true";
        flag_c[$min-$i+$size-1]="true";
        N-Queen4_rec "$((min+1))" "$size";
        flag_a[$i]="";
        flag_b[$min+$i]="";
        flag_c[$min-$i+$size-1]="";
      }
    }
  }
}
#
function N-Queen4(){
  local -i max=8;
  local -i min=2;
  local -i N="$min";
  local startTime=0;
  local endTime=0;
  local hh=mm=ss=0;
  echo " N:        Total       Unique        hh:mm:ss";
  for((N=min;N<=max;N++)){
    TOTAL=0;
    UNIQUE=0;
    startTime=$(date +%s);
    for((k=0;k<N;k++)){
      board[k]=k;
    }
    N-Queen4_rec 0 "$N";
    endTime=$(date +%s);
    ss=$((endTime-startTime));
    hh=$((ss/3600));
    ss=$((ss/3600));
    mm=$((ss/60));
    ss=$((ss%60));
    printf "%2d:%13d%13d%10d:%.2d:%.2d\n" $N $TOTAL $UNIQUE $hh $mm $ss;
  }

}
#
echo "<>4.BT+対象解除法 N-Queen4()";
N-Queen4;
#


<>4.BT+対象解除法 N-Queen4()
 N:        Total       Unique        hh:mm:ss
 2:            0            0         0:00:00
 3:            0            0         0:00:00
 4:            2            1         0:00:00
 5:           10            2         0:00:00
 6:            4            1         0:00:00
 7:           40            6         0:00:00
 8:           92           12         0:00:00


# Bash05

In [ ]:
%%bash

typeset -i TOTAL=0;
typeset -i UNIQUE=0;
typeset -a flag_a=""; 
typeset -a flag_b="";
typeset -a flag_c="";
typeset -a trial="";
typeset -a board="";
typeset -a scratch="";
#
function intncmp(){
  local -i k; 
  local -i rtn=0;
  local -i n=$1;
  for((k=0;k<n;k++)){
    rtn=$((board[k]-trial[k]));
    ((rtn!=0))&&{ break; }
  }
  echo "$rtn";
}
#
function rotate() {
  local -i j;
  local -i k;
  local -i n=$1;
  local -i incr;
  local neg=$2;
  if [ "$neg" = "true" ];then
    k=0;
  else
    k=$((n-1)); 
  fi 
  if [ "$neg" = "true" ];then
    incr=$((incr+1));
  else
    incr=$((incr-1));
  fi 
  for((j=0;j<n;k+=incr)){ 
    j=$((j+1))
    scratch[$j]=${trial[$k]};
  }
  if [ "$neg" = "true" ];then
    k=$((n-1));
  else
    k=0;
  fi 
  for((j=0;j<n;k-=incr)){ 
    j=$((j+1))
    trial[${scratch[$j]}]=$k;
  }
}
#
function vMirror(){
  local -i j;
  local -i n=$1;
  for((j=0;j<n;j++)){
    local -i n1=$((n-1));
    trial[$j]=$((n1-trial[j]));
  }
}
#
function symmetryOps() {
  local -i k;
  local -i nEquiv;
  local -i size=$1;
  
  for((k=0;k<size;k++)){
    trial[$k]=${board[$k]};
  }
  rotate "$size" "false";
  k=$(intncmp "$size");
  ((k>0))&&{
    echo 0; 
    return;
  }  
  ((k==0))&&{
     nEquiv=1;
  }||{
     rotate "$size" "false";
     k=$(intncmp "$size");
     ((k>0))&&{
       echo 0; 
       return;
     }
     ((k==0))&&{
        nEquiv=2;
   }||{
      rotate "$size" "false";
      k=$(intncmp "$size");
      ((k>0))&&{
        echo 0; 
        return;
      }  
      nEquiv=4;
     }
  }
  for((k=0;k<size;k++)){ 
    trial[$k]=${board[$k]};
  }
  vMirror "$size";
  k=$(intncmp "$size");
  ((k>0))&&{ 
    echo 0; 
    return;
  }
  ((nEquiv>1))&&{
     rotate "$size" "true";
     k=$(intncmp "$size");
     ((k>0))&&{
       echo 0;
       return;
     }
     ((nEquiv>2))&&{
        rotate "$size" "true";
        k=$(intncmp "$size");
        ((k>0))&&{ 
          echo 0;
          return;
        }
        rotate "$size" "true";
        k=$(intncmp "$size");
        ((k>0))&&{
          echo 0;
          return;
        }
     }
  }
  rtn=$((nEquiv * 2));
  echo "$rtn";
  return;
 }
#
function N-Queen5_rec(){
  local -i min="$1";
  local -i size="$2";
  local -i i=0;
  ((min != 0))&&{
    lim=$size;
  }||{
    lim=$(((size+1)/2)); 
  }
  for((i=0;i<lim;i++)){
    [ "${flag_a[$i]}" != "true"  ]&& \
    [ "${flag_b[$min+$i]}" != "true" ]&& \
    [ "${flag_c[$min-$i+$size-1]}" != "true" ]&&{
      board[$min]=$i ;
      ((min==(size-1)))&&{
        tst=$(symmetryOps "$size");
        ((tst!=0))&&{
          ((UNIQUE++));
          TOTAL=$((TOTAL+tst));
        }
      }||{
        flag_a[$i]="true";
        flag_b[$min+$i]="true"; 
        flag_c[$min-$i+$size-1]="true"; 
        N-Queen5_rec "$((min+1))" "$size";
        flag_a[$i]="";           
        flag_b[$min+$i]="";   
        flag_c[$min-$i+$size-1]=""; 
      }          
    }
  }  
}
#
function N-Queen5(){
  local -i max=8;
  local -i min=2;
  local -i N="$min";
  local startTime=0;
  local endTime=0;
  local hh=mm=ss=0;
  echo " N:        Total       Unique        hh:mm:ss";
  for((N=min;N<=max;N++)){
    TOTAL=0;
    UNIQUE=0;
    startTime=$(date +%s);
    for((k=0;k<N;k++)){ board[$k]=$k;}
    N-Queen5_rec 0 "$N";
    endTime=$(date +%s);
    ss=$((endTime-startTime));
    hh=$((ss/3600));
    ss=$((ss%3600));
    mm=$((ss/60));
    ss=$((ss%60));
    printf "%2d:%13d%13d%10d:%.2d.%.2d\n" $N $TOTAL $UNIQUE $hh $mm $ss;
  }
}
#
echo "<>5.BT＋対象解除法＋枝刈り N-Queen5()";
N-Queen5 ;
#

<>5.BT＋対象解除法＋枝刈り N-Queen5()
 N:        Total       Unique        hh:mm:ss
 2:            0            0         0:00.00
 3:            0            0         0:00.00
 4:            2            1         0:00.00
 5:           10            2         0:00.00
 6:            4            1         0:00.00
 7:           40            6         0:00.00
 8:           92           12         0:00.01


# Bash06

In [ ]:
%%bash

typeset -i TOTAL=0;
typeset -i UNIQUE=0;
typeset -i size=0;
typeset -i MASK=0;
#
function N-Queen6_rec(){
  local -i min="$1";
	local -i left="$2";
	local -i down="$3";
	local -i right="$4";
	local -i bitmap=;
	local -i bit=;
  ((min==size))&&((TOTAL++))||{
    bitmap=$((MASK&~(left|down|right)));
    while ((bitmap)); do
      bit=$((-bitmap&bitmap)) ;
      bitmap=$((bitmap^bit)) ;
      N-Queen6_rec "$((min+1))" "$(((left|bit)<<1))" "$((down|bit))" "$(((right|bit)>>1))"  ;
    done
  }
}
#
function N-Queen6(){
  local -i max=10;
	local -i min=2;
	local startTime=;
	local endTime= ;
	local hh=mm=ss=0;
  echo " N:        Total       Unique        hh:mm:ss" ;
  for ((size=min;size<=max;size++)) {
    TOTAL=0;
		UNIQUE=0;
		MASK=$(((1<<size)-1));
    startTime=$(date +%s);
    N-Queen6_rec 0 0 0 0 ;
    endTime=$(date +%s);
    ss=$((endTime-startTime));
    hh=$((ss/3600));
    ss=$((ss%3600));
    mm=$((ss/60));
    ss=$((ss%60));
    printf "%2d:%13d%13d%10d:%.2d:%.2d\n" $size $TOTAL $UNIQUE $hh $mm $ss ;
  } 
}
#
  echo "<>６．BT＋ビットマップ(Bit) N-Queen6()";
  N-Queen6;
#

<>６．BT＋ビットマップ(Bit) N-Queen6()
 N:        Total       Unique        hh:mm:ss
 2:            0            0         0:00:00
 3:            0            0         0:00:00
 4:            2            0         0:00:00
 5:           10            0         0:00:00
 6:            4            0         0:00:00
 7:           40            0         0:00:00
 8:           92            0         0:00:00
 9:          352            0         0:00:00
10:          724            0         0:00:02


# Bash07

In [ ]:
%%bash

typeset -i TOTAL=0;
typeset -i UNIQUE=0;
typeset -i size=0;
typeset -i MASK=0;
typeset -a board="";
typeset -a trial="";
typeset -a scratch="";
typeset -i COUNT2=0;
typeset -i COUNT4=0;
typeset -i COUNT8=0;
#
function getUnique(){ 
  echo $((COUNT2+COUNT4+COUNT8));
}
#
function getTotal(){ 
  echo $((COUNT2*2+COUNT4*4+COUNT8*8));
}
#
function rotate_bitmap_ts(){
  local -i t=0;
  for((i=0;i<size;i++)){
    t=0;
    for((j=0;j<size;j++)){
      ((t|=((trial[j]>>i)&1)<<(size-j-1))); 
    }
    scratch[$i]=$t; 
  }
}
#
function rotate_bitmap_st(){
  local -i t=0;
  for((i=0;i<size;i++)){
    t=0;
    for((j=0;j<size;j++)){
      ((t|=((scratch[j]>>i)&1)<<(size-j-1))); 
    }
    trial[$i]=$t; 
  }
}
#
function rh(){
  local -i a=$1;
  local -i sz=$2;
  local -i tmp=0;
  for((i=0;i<=sz;i++)){
    ((a&(1<<i)))&&{ 
     (( tmp|=(1<<(sz-i)) )); 
    }
  }
  echo $tmp;
}
#
function vMirror_bitmap(){
  local -i score=0;
  local -i sizeE=$((size-1));
  for((i=0;i<size;i++)){
    score=${scratch[$i]};
    trial[$i]=$(rh "$score" $sizeE);
  }
}
#
function intncmp_bs(){
  local -i rtn=0;
  for((i=0;i<size;i++)){
    rtn=$(echo "${board[$i]}-${scratch[$i]}"+10);
    ((rtn!=10))&&{ break; }
  }
  echo "$rtn";
}
#
function intncmp_bt(){
  local -i rtn=0;
  for((i=0;i<size;i++)){
    rtn=$(echo "${board[$i]}-${trial[$i]}"+10);
    ((rtn!=10))&&{ break; }
  }
  echo "$rtn";
}
#
function symmetryOps_bm(){
  local -i nEquiv=0;
  #回転・反転・対称チェックのためにboard配列をコピー
  for((i=0;i<size;i++)){ 
    trial[$i]=${board[$i]};
  }
  rotate_bitmap_ts; 
  #    //時計回りに90度回転
  k=$(intncmp_bs);
  ((k>10))&&{ 
   return; 
  }
  ((k==10))&&{ 
    nEquiv=2;
  }||{
    rotate_bitmap_st;
    #  //時計回りに180度回転
    k=$(intncmp_bt);
    ((k>10))&&{ 
     return; 
    }
    ((k==10))&&{ 
      nEquiv=4;
    }||{
      rotate_bitmap_ts;
      #//時計回りに270度回転
      k=$(intncmp_bs);
      ((k>10))&&{ 
        return;
      }
      nEquiv=8;
    }
  }
  #// 回転・反転・対称チェックのためにboard配列をコピー
  for((i=0;i<size;i++)){ 
    scratch[$i]=${board[$i]};
  }
  vMirror_bitmap;
  #//垂直反転
  k=$(intncmp_bt);
  ((k>10))&&{ 
   return; 
  }
  ((nEquiv>2))&&{
    #               //-90度回転 対角鏡と同等       
    rotate_bitmap_ts;
    k=$(intncmp_bs);
    ((k>10))&&{
      return;
    }
    ((nEquiv>4))&&{
      #             //-180度回転 水平鏡像と同等
      rotate_bitmap_st;
      k=$(intncmp_bt);
      #((k>0))&&{ 
      ((k>10))&&{ 
        return;
      } 
      #             //-270度回転 反対角鏡と同等
      rotate_bitmap_ts;
      k=$(intncmp_bs);
      ((k>10))&&{ 
        return;
      }
    }
  }
  ((nEquiv==2))&&{
    ((COUNT2++));
  }
  ((nEquiv==4))&&{
    ((COUNT4++));
  }
  ((nEquiv==8))&&{
    ((COUNT8++));
  }
}
#
function N-Queen7_rec(){
	#y: l:left d:down r:right b:bit bm:bitmap
  local -i min="$1";
	local -i left="$2";
	local -i down="$3";
	local -i right="$4";
	local -i bitmap=0;
  bitmap=$((MASK&~(left|down|right)));
  ((min==size&&!bitmap))&&{
    board[$min]=$bitmap;
    symmetryOps_bm;
	}||{
    while ((bitmap)); do
      bit=$((-bitmap&bitmap)) ;
      board[$min]=$bit;
      bitmap=$((bitmap^bit)) ;
      N-Queen7_rec "$((min+1))" "$(((left|bit)<<1))" "$((down|bit))" "$(((right|bit)>>1))"  ;
    done
  }
}
#
function N-Queen7(){
  local -i max=10;
	local -i min=2;
	local startTime=;
	local endTime= ;
	local hh=mm=ss=0;
  echo " N:        Total       Unique        hh:mm:ss" ;
  for ((size=min;size<=max;size++)) {
    TOTAL=0;
		UNIQUE=0;
    COUNT2=COUNT4=COUNT8=0;
    for((j=0;j<size;j++)){
     board[$j]=$j; 
    }
		MASK=$(((1<<size)-1));
    startTime=$(date +%s);
    N-Queen7_rec 0 0 0 0 ;
    endTime=$(date +%s);
    ss=$((endTime-startTime));
    hh=$((ss/3600));
    ss=$((ss%3600));
    mm=$((ss/60));
    ss=$((ss%60));
    TOTAL=$(getTotal);
    UNIQUE=$(getUnique);
    printf "%2d:%13d%13d%10d:%.2d:%.2d\n" $size $TOTAL $UNIQUE $hh $mm $ss ;
  } 
}
#
  echo "<>７．BT＋Bit＋対称解除法 N-Queen7()";
  N-Queen7;
#

<>７．BT＋Bit＋対称解除法 N-Queen7()
 N:        Total       Unique        hh:mm:ss
 2:            0            0         0:00:00
 3:            0            0         0:00:00
 4:            2            1         0:00:00
 5:           10            2         0:00:00
 6:            4            1         0:00:00
 7:           40            6         0:00:00
 8:           92           12         0:00:01
 9:          352           46         0:00:04
10:          724           92         0:00:09


# Bash08

In [ ]:
%%bash

typeset -i TOTAL=0;
typeset -i UNIQUE=0;
typeset -i size=0;
typeset -i MASK=0;
typeset -a board="";
typeset -a trial="";
typeset -a scratch="";
typeset -a flag_a="";
typeset -a flag_b="";
typeset -a flag_c="";
typeset -i COUNT2=0;
typeset -i COUNT4=0;
typeset -i COUNT8=0;
#
function getUnique(){ 
  echo $((COUNT2+COUNT4+COUNT8));
}
#
function getTotal(){ 
  echo $((COUNT2*2+COUNT4*4+COUNT8*8));
}
#
function rotate_bitmap_ts(){
  local -i t=0;
  for((i=0;i<size;i++)){
    t=0;
    for((j=0;j<size;j++)){
      ((t|=((trial[j]>>i)&1)<<(size-j-1))); 
    }
    scratch[$i]=$t; 
  }
}
#
function rotate_bitmap_st(){
  local -i t=0;
  for((i=0;i<size;i++)){
    t=0;
    for((j=0;j<size;j++)){
      ((t|=((scratch[j]>>i)&1)<<(size-j-1))); 
    }
    trial[$i]=$t; 
  }
}
#
function rh(){
  local -i a=$1;
  local -i sz=$2;
  local -i tmp=0;
  for((i=0;i<=sz;i++)){
    ((a&(1<<i)))&&{ 
     ((tmp|=(1<<(sz-i)))); 
    }
  }
  echo $tmp;
}
#
function vMirror_bitmap(){
  local -i score=0;
  local -i sizeE=$((size-1));
  for((i=0;i<size;i++)){
    score=${scratch[$i]};
    trial[$i]=$(rh "$score" $sizeE);
  }
}
#
function intncmp_bs(){
  local -i rtn=0;
  for((i=0;i<size;i++)){
    rtn=$(echo "${board[$i]}-${scratch[$i]}"+10);
    ((rtn!=10))&&{ break; }
  }
  echo "$rtn";
}
#
function intncmp_bt(){
  local -i rtn=0;
  for((i=0;i<size;i++)){
    #rtn=$(echo "${board[$i]}-${trial[$i]}"|bc);
    rtn=$(echo "${board[$i]}-${trial[$i]}"+10);
    #((rtn!=0))&&{ break; }
    ((rtn!=10))&&{ break; }
  }
  echo "$rtn";
}
#
function symmetryOps_bm(){
#  local -i si=$1;
  local -i nEquiv=0;
  #回転・反転・対称チェックのためにboard配列をコピー
  for((i=0;i<size;i++)){ 
    trial[$i]=${board[$i]};
  }
  #rotate_bitmap_ts "$size";
  rotate_bitmap_ts; 
  #    //時計回りに90度回転
  k=$(intncmp_bs);
  #((k>0))&&{ 
  ((k>10))&&{ 
   return; 
  }
  #((k==0))&&{ 
  ((k==10))&&{ 
    nEquiv=2;
  }||{
    rotate_bitmap_st;
    #  //時計回りに180度回転
    k=$(intncmp_bt);
    #((k>0))&&{ 
    ((k>10))&&{ 
     return; 
    }
    #((k==0))&&{ 
    ((k==10))&&{ 
      nEquiv=4;
    }||{
      rotate_bitmap_ts;
      #//時計回りに270度回転
      k=$(intncmp_bs);
      #((k>0))&&{ 
      ((k>10))&&{ 
        return;
      }
      nEquiv=8;
    }
  }
  #// 回転・反転・対称チェックのためにboard配列をコピー
  for((i=0;i<size;i++)){ 
    scratch[$i]=${board[$i]};
  }
  vMirror_bitmap;
  #//垂直反転
  k=$(intncmp_bt);
  #((k>0))&&{ 
  ((k>10))&&{ 
   return; 
  }
  ((nEquiv>2))&&{
  #               //-90度回転 対角鏡と同等       
    rotate_bitmap_ts;
    k=$(intncmp_bs);
    #((k>0))&&{
    ((k>10))&&{
      return;
    }
    ((nEquiv>4))&&{
    #             //-180度回転 水平鏡像と同等
      #rotate_bitmap_st "$size";
      rotate_bitmap_st;
      k=$(intncmp_bt);
      #((k>0))&&{ 
      ((k>10))&&{ 
        return;
      } 
      #      //-270度回転 反対角鏡と同等
      rotate_bitmap_ts;
      k=$(intncmp_bs);
      #((k>0))&&{ 
      ((k>10))&&{ 
        return;
      }
    }
  }
  ((nEquiv==2))&&{
    ((COUNT2++));
  }
  ((nEquiv==4))&&{
    ((COUNT4++));
  }
  ((nEquiv==8))&&{
    ((COUNT8++));
  }
}
#
function N-Queen8_rec(){
  local -i min="$1";
	local -i left="$2";
	local -i down="$3";
	local -i right="$4";
	local -i bitmap=0;
  bitmap=$((MASK&~(left|down|right)));
  ((min==size&&!bitmap))&&{
    board[$min]=$bitmap;
    symmetryOps_bm;
	}||{
 		# 枝刈り
 		((min!=0))&&{
 			lim=$size;
 		}||{
 			lim=$(((size+1)/2)); 
 		}
 		for((s=min+1;s<lim,bitmap;s++)){
		# 枝刈りによりwhileは不要
    # while ((bitmap)); do
      bit=$((-bitmap&bitmap)) ;
      board[$min]=$bit;
      bitmap=$((bitmap^bit)) ;
      N-Queen8_rec "$((min+1))" "$(((left|bit)<<1))" "$((down|bit))" "$(((right|bit)>>1))"  ;
		# 枝刈りによりwhileは不要
		}
	}
}
#
function N-Queen8(){
  local -i max=10;
	local -i min=2;
	local startTime=;
	local endTime= ;
	local hh=mm=ss=0; 		# いっぺんにに初期化することもできます
  echo " N:        Total       Unique        hh:mm:ss" ;
  for ((size=min;size<=max;size++)) {
    TOTAL=0;
		UNIQUE=0;
    COUNT2=COUNT4=COUNT8=0;
    for((j=0;j<$size;j++)){
     board[$j]=$j; 
    }
		MASK=$(((1<<size)-1));
    startTime=$(date +%s);# 計測開始時間
    N-Queen8_rec 0 0 0 0 ;
    endTime=$(date +%s); 	# 計測終了時間
    ss=$((endTime-startTime));# hh:mm:ss 形式に変換
    hh=$((ss/3600));
    ss=$((ss%3600));
    mm=$((ss/60));
    ss=$((ss%60));
    TOTAL=$(getTotal);
    UNIQUE=$(getUnique);
    printf "%2d:%13d%13d%10d:%.2d:%.2d\n" $size $TOTAL $UNIQUE $hh $mm $ss ;
  } 
}
#
#
  echo "<>８．BT＋Bit＋対称解除法＋枝刈り N-Queen8()";
  N-Queen8;
#

<>８．BT＋Bit＋対称解除法＋枝刈り N-Queen8()
 N:        Total       Unique        hh:mm:ss
 2:            0            0         0:00:00
 3:            0            0         0:00:00
 4:            2            1         0:00:00
 5:           10            2         0:00:00
 6:            4            1         0:00:00
 7:           40            6         0:00:00
 8:           92           12         0:00:01
 9:          352           46         0:00:04
10:          724           92         0:00:10


# Bash09

In [ ]:
%%bash

typeset -i TOTAL=0;
typeset -i UNIQUE=0;
typeset -i size=0;
typeset -i MASK=0;
typeset -i BOUND1=0;
typeset -i BOUND2=0;
typeset -i TOPBIT=0;
typeset -i ENDBIT=0;
typeset -i SIDEMASK=0;
typeset -i LASTMASK=0;
typeset -a board="";
typeset -a trial="";
typeset -a scratch="";
typeset -a flag_a="";     # -a は配列の型を宣言します
typeset -a flag_b="";
typeset -a flag_c="";
typeset -i COUNT2=0;
typeset -i COUNT4=0;
typeset -i COUNT8=0;
#
function getUnique(){ 
  echo $((COUNT2+COUNT4+COUNT8));
}
#
function getTotal(){ 
  echo $(( COUNT2*2 + COUNT4*4 + COUNT8*8));
}
#
function rotate_bitmap_ts(){
  local -i t=0;
  for((i=0;i<size;i++)){
    t=0;
    for((j=0;j<size;j++)){
      ((t|=((trial[j]>>i)&1)<<(size-j-1))); 
    }
    scratch[$i]=$t; 
  }
}
#
function rotate_bitmap_st(){
  local -i t=0;
  for((i=0;i<size;i++)){
    t=0;
    for((j=0;j<size;j++)){
      ((t|=((scratch[j]>>i)&1)<<(size-j-1))); 
    }
    trial[$i]=$t; 
  }
}
#
function rh(){
  local -i a=$1;
  local -i sz=$2;
  local -i tmp=0;
  for((i=0;i<=sz;i++)){
    ((a&(1<<i)))&&{ 
     ((tmp|=(1<<(sz-i)))); 
    }
  }
  echo $tmp;
}
#
function vMirror_bitmap(){
  local -i score=0;
  local -i sizeE=$((size-1));
  for((i=0;i<size;i++)){
    score=${scratch[$i]};
    trial[$i]=$(rh "$score" $sizeE);
  }
}
function intncmp_bs(){
  local -i rtn=0;
  for((i=0;i<size;i++)){
    rtn=$(echo "${board[$i]}-${scratch[$i]}"+10);
    ((rtn!=10))&&{ break; }
  }
  echo "$rtn";
}
function intncmp_bt(){
  local -i rtn=0;
  for((i=0;i<size;i++)){
    rtn=$(echo "${board[$i]}-${trial[$i]}"+10);
    ((rtn!=10))&&{ break; }
  }
  echo "$rtn";
}
#
function symmetryOps_bm(){
  local -i nEquiv=0;
  #回転・反転・対称チェックのためにboard配列をコピー
  for((i=0;i<size;i++)){ 
    trial[$i]=${board[$i]};
  }
  rotate_bitmap_ts; 
  #    //時計回りに90度回転
  k=$(intncmp_bs);
  ((k>10))&&{ 
   return; 
  }
  ((k==10))&&{ 
    nEquiv=2;
  }||{
    rotate_bitmap_st;
    #  //時計回りに180度回転
    k=$(intncmp_bt);
    ((k>10))&&{ 
     return; 
    }
    ((k==10))&&{ 
      nEquiv=4;
    }||{
      rotate_bitmap_ts;
      #//時計回りに270度回転
      k=$(intncmp_bs);
      ((k>10))&&{ 
        return;
      }
      nEquiv=8;
    }
  }
  #// 回転・反転・対称チェックのためにboard配列をコピー
  for((i=0;i<size;i++)){ 
    scratch[$i]=${board[$i]};
  }
  vMirror_bitmap;
  #//垂直反転
  k=$(intncmp_bt);
  ((k>10))&&{ 
   return; 
  }
  ((nEquiv>2))&&{
  #               //-90度回転 対角鏡と同等       
    rotate_bitmap_ts;
    k=$(intncmp_bs);
    ((k>10))&&{
      return;
    }
    ((nEquiv>4))&&{
    #             //-180度回転 水平鏡像と同等
      rotate_bitmap_st;
      k=$(intncmp_bt);
      ((k>10))&&{ 
        return;
      } 
      #      //-270度回転 反対角鏡と同等
      rotate_bitmap_ts;
      k=$(intncmp_bs);
      ((k>10))&&{ 
        return;
      }
    }
  }
  ((nEquiv==2))&&{
    ((COUNT2++));
  }
  ((nEquiv==4))&&{
    ((COUNT4++));
  }
  ((nEquiv==8))&&{
    ((COUNT8++));
  }
}
#
function backTrack(){
	local -i bit;
  local -i min="$1";
	local -i left="$2";
	local -i down="$3";
	local -i right="$4";
	local -i bitmap=0;
  bitmap=$((MASK&~(left|down|right)));
	((min==size&&!bitmap))&&{
	  board[$min]=bitmap;
    symmetryOps_bm;
	}||{
		while((bitmap));do
      bit=$((-bitmap&bitmap)) ;
      board[$min]=$bit;
      bitmap=$((bitmap^bit)) ;
      backTrack "$((min+1))" "$(((left|bit)<<1))" "$((down|bit))" "$(((right|bit)>>1))"  ;
		done
	}
}
#
function N-Queen9_rec(){
 	local -i min="$1";
	((TOPBIT=1<<(size-1)));
  LASTMASK=$((TOPBIT|1));
	SIDEMASK=$LASTMASK;
	ENDBIT=$((TOPBIT>>1));
	BOUND2=$((size-2));
	for ((BOUND1=0;BOUND1<BOUND2;BOUND1++)){
	  bit=$((1<<BOUND1));
	  board[0]=$bit;
    backTrack "1" "$((bit<<1))" "$((bit))" "$((bit>>1))";
	  ((LASTMASK|=LASTMASK>>1|LASTMASK<<1));
	  ((ENDBIT>>=1));
	  ((BOUND2--));
	}
}
#
function N-Queen9(){
  local -i max=10;
	local -i min=2;
	local startTime=;
	local endTime= ;
	local hh=mm=ss=0; 		# いっぺんにに初期化することもできます
  echo " N:        Total       Unique        hh:mm:ss" ;
  for ((size=min;size<=max;size++)) {
    TOTAL=0;
		UNIQUE=0;
    COUNT2=COUNT4=COUNT8=0;
    for((j=0;j<$size;j++)){
     board[$j]=$j; 
    }
		MASK=$(((1<<size)-1));
    startTime=$(date +%s);# 計測開始時間
    N-Queen9_rec 0;
    endTime=$(date +%s); 	# 計測終了時間
    ss=$((endTime-startTime));# hh:mm:ss 形式に変換
    hh=$((ss/3600));
    ss=$((ss%3600));
    mm=$((ss/60));
    ss=$((ss%60));
    TOTAL=$(getTotal);
    UNIQUE=$(getUnique);
    printf "%2d:%13d%13d%10d:%.2d:%.2d\n" $size $TOTAL $UNIQUE $hh $mm $ss ;
  } 
}
#
  echo "<>９．BT＋Bit＋対称解除Bit＋クイーンの位置による振り分け(BOUND1) N-Queen9()";
  N-Queen9;
#

<>９．BT＋Bit＋対称解除Bit＋クイーンの位置による振り分け(BOUND1) N-Queen9()
 N:        Total       Unique        hh:mm:ss
 2:            0            0         0:00:00
 3:            0            0         0:00:00
 4:            0            0         0:00:00
 5:           10            2         0:00:00
 6:            4            1         0:00:00
 7:           40            6         0:00:01
 8:           92           12         0:00:00
 9:          352           46         0:00:03
10:          724           92         0:00:06


# Bash10

In [ ]:
%%bash

typeset -i TOTAL=0;
typeset -i UNIQUE=0;
typeset -i size=0;
typeset -i MASK=0;
typeset -i BOUND1=0;
typeset -i BOUND2=0;
typeset -i TOPBIT=0;
typeset -i ENDBIT=0;
typeset -i SIDEMASK=0;
typeset -i LASTMASK=0;
typeset -a board="";
typeset -a trial="";
typeset -a scratch="";
typeset -a flag_a="";     # -a は配列の型を宣言します
typeset -a flag_b="";
typeset -a flag_c="";
typeset -i COUNT2=0;
typeset -i COUNT4=0;
typeset -i COUNT8=0;
#
function getUnique(){ 
  echo $((COUNT2+COUNT4+COUNT8));
}
#
function getTotal(){ 
  echo $(( COUNT2*2 + COUNT4*4 + COUNT8*8));
}
#
function rotate_bitmap_ts(){
  local -i t=0;
  for((i=0;i<size;i++)){
    t=0;
    for((j=0;j<size;j++)){
      ((t|=((trial[j]>>i)&1)<<(size-j-1))); 
    }
    scratch[$i]=$t; 
  }
}
#
function rotate_bitmap_st(){
  local -i t=0;
  for((i=0;i<size;i++)){
    t=0;
    for((j=0;j<size;j++)){
      ((t|=((scratch[j]>>i)&1)<<(size-j-1))); 
    }
    trial[$i]=$t; 
  }
}
#
function rh(){
  local -i a=$1;
  local -i sz=$2;
  local -i tmp=0;
  for((i=0;i<=sz;i++)){
    ((a&(1<<i)))&&{ 
     ((tmp|=(1<<(sz-i)))); 
    }
  }
  echo $tmp;
}
#
function vMirror_bitmap(){
  local -i score=0;
  local -i sizeE=$((size-1));
  for((i=0;i<size;i++)){
    score=${scratch[$i]};
    trial[$i]=$(rh "$score" $sizeE);
  }
}
#
function intncmp_bs(){
  local -i rtn=0;
  for((i=0;i<size;i++)){
    rtn=$(echo "${board[$i]}-${scratch[$i]}"+10);
    ((rtn!=10))&&{ break; }
  }
  echo "$rtn";
}
#
function intncmp_bt(){
  local -i rtn=0;
  for((i=0;i<size;i++)){
    rtn=$(echo "${board[$i]}-${trial[$i]}"+10);
    ((rtn!=10))&&{ break; }
  }
  echo "$rtn";
}
#
function symmetryOps_bm(){
  local -i nEquiv=0;
  #回転・反転・対称チェックのためにboard配列をコピー
  for((i=0;i<size;i++)){ 
    trial[$i]=${board[$i]};
  }
  rotate_bitmap_ts; 
  #    //時計回りに90度回転
  k=$(intncmp_bs);
  ((k>10))&&{ 
   return; 
  }
  ((k==10))&&{ 
    nEquiv=2;
  }||{
    rotate_bitmap_st;
    #  //時計回りに180度回転
    k=$(intncmp_bt);
    ((k>10))&&{ 
     return; 
    }
    ((k==10))&&{ 
      nEquiv=4;
    }||{
      rotate_bitmap_ts;
      #//時計回りに270度回転
      k=$(intncmp_bs);
      ((k>10))&&{ 
        return;
      }
      nEquiv=8;
    }
  }
  #// 回転・反転・対称チェックのためにboard配列をコピー
  for((i=0;i<size;i++)){ 
    scratch[$i]=${board[$i]};
  }
  vMirror_bitmap;
  #//垂直反転
  k=$(intncmp_bt);
  ((k>10))&&{ 
   return; 
  }
  ((nEquiv>2))&&{
  #               //-90度回転 対角鏡と同等       
    rotate_bitmap_ts;
    k=$(intncmp_bs);
    ((k>10))&&{
      return;
    }
    ((nEquiv>4))&&{
    #             //-180度回転 水平鏡像と同等
      rotate_bitmap_st;
      k=$(intncmp_bt);
      ((k>10))&&{ 
        return;
      } 
      #      //-270度回転 反対角鏡と同等
      rotate_bitmap_ts;
      k=$(intncmp_bs);
      ((k>10))&&{ 
        return;
      }
    }
  }
  ((nEquiv==2))&&{
    ((COUNT2++));
  }
  ((nEquiv==4))&&{
    ((COUNT4++));
  }
  ((nEquiv==8))&&{
    ((COUNT8++));
  }
}
#
function backTrack2(){
	local -i bit;
  local -i min="$1";
	local -i left="$2";
	local -i down="$3";
	local -i right="$4";
	local -i bitmap=0;
  bitmap=$((MASK&~(left|down|right)));
	((min==size&&!bitmap))&&{
	  board[$min]=bitmap;
    symmetryOps_bm;
	}||{
		while((bitmap));do
			bit=$((-bitmap&bitmap)) ;
			board[$min]=$bit;
			bitmap=$((bitmap^bit)) ;
			backTrack2 "$((min+1))" "$(((left|bit)<<1))" "$((down|bit))" "$(((right|bit)>>1))"  ;
		done
	}
}
#
function backTrack1(){
	local -i bit;
  local -i min="$1";
	local -i left="$2";
	local -i down="$3";
	local -i right="$4";
	local -i bitmap=0;
  bitmap=$((MASK&~(left|down|right)));
	((min==size&&!bitmap))&&{
	  board[$min]=bitmap;
    symmetryOps_bm;
	}||{
		while((bitmap));do
			bit=$((-bitmap&bitmap)) ;
			board[$min]=$bit;
			bitmap=$((bitmap^bit)) ;
			backTrack1 "$((min+1))" "$(((left|bit)<<1))" "$((down|bit))" "$(((right|bit)>>1))"  ;
		done
	}
}
#
function N-Queen10_rec(){
 	local -i min="$1";
	((TOPBIT=1<<(size-1)));
	board[0]=1;
	for((BOUND1=2;BOUND1<size-1;BOUND1++)){
	  bit=$((1<<BOUND1));
	  board[1]=$bit;
    backTrack1 "2" "$(((2|bit)<<1))" "$(((1|bit)))" "$((bit>>1))";
	}

  LASTMASK=$((TOPBIT|1));
	SIDEMASK=$LASTMASK;
	ENDBIT=$((TOPBIT>>1));
	BOUND2=$((size-2));
	for ((BOUND1=1;BOUND1<BOUND2;BOUND1++)){
	  bit=$((1<<BOUND1));
	  board[0]=$bit;
    backTrack2 "1" "$((bit<<1))" "$((bit))" "$((bit>>1))";
	  ((LASTMASK|=LASTMASK>>1|LASTMASK<<1));
	  ((ENDBIT>>=1));
	  ((BOUND2--));
	}
}
function N-Queen10(){
  local -i max=10;
	local -i min=2;
	local startTime=;
	local endTime= ;
	local hh=mm=ss=0; 		# いっぺんにに初期化することもできます
  echo " N:        Total       Unique        hh:mm:ss" ;
  for ((size=min;size<=max;size++)) {
    TOTAL=0;
		UNIQUE=0;
    COUNT2=COUNT4=COUNT8=0;
    for((j=0;j<$size;j++)){
     board[$j]=$j; 
    }
		MASK=$(((1<<size)-1));
    startTime=$(date +%s);# 計測開始時間
    N-Queen10_rec 0;
    endTime=$(date +%s); 	# 計測終了時間
    ss=$((endTime-startTime));# hh:mm:ss 形式に変換
    hh=$((ss/3600));
    ss=$((ss%3600));
    mm=$((ss/60));
    ss=$((ss%60));
    TOTAL=$(getTotal);
    UNIQUE=$(getUnique);
    printf "%2d:%13d%13d%10d:%.2d:%.2d\n" $size $TOTAL $UNIQUE $hh $mm $ss ;
  } 
}
#
  echo "<>１０．BT＋Bit＋対称解除Bit＋クイーンの位置による振り分け (BOUND1+BOUND2) N-Queen10()";
  N-Queen10;
#

<>１０．BT＋Bit＋対称解除Bit＋クイーンの位置による振り分け (BOUND1+BOUND2) N-Queen10()
 N:        Total       Unique        hh:mm:ss
 2:            0            0         0:00:00
 3:            0            0         0:00:00
 4:            2            1         0:00:00
 5:           10            2         0:00:00
 6:            4            1         0:00:00
 7:           40            6         0:00:01
 8:           92           12         0:00:00
 9:          352           46         0:00:03
10:          724           92         0:00:07


# Bash11

In [ ]:
%%bash

typeset -i TOTAL=0;
typeset -i UNIQUE=0;
typeset -i size=0;
typeset -i MASK=0;
typeset -i BOUND1=0;
typeset -i BOUND2=0;
typeset -i TOPBIT=0;
typeset -i ENDBIT=0;
typeset -i SIDEMASK=0;
typeset -i LASTMASK=0;
typeset -a board="";
typeset -a trial="";
typeset -a scratch="";
typeset -a flag_a="";     # -a は配列の型を宣言します
typeset -a flag_b="";
typeset -a flag_c="";
typeset -i COUNT2=0;
typeset -i COUNT4=0;
typeset -i COUNT8=0;
#
function getUnique(){ 
  echo $((COUNT2+COUNT4+COUNT8));
}
#
function getTotal(){ 
  echo $(( COUNT2*2 + COUNT4*4 + COUNT8*8));
}
#
function rotate_bitmap_ts(){
  local -i t=0;
  for((i=0;i<size;i++)){
    t=0;
    for((j=0;j<size;j++)){
      ((t|=((trial[j]>>i)&1)<<(size-j-1))); 
    }
    scratch[$i]=$t; 
  }
}
#
function rotate_bitmap_st(){
  local -i t=0;
  for((i=0;i<size;i++)){
    t=0;
    for((j=0;j<size;j++)){
      ((t|=((scratch[j]>>i)&1)<<(size-j-1))); 
    }
    trial[$i]=$t; 
  }
}
#
function rh(){
  local -i a=$1;
  local -i sz=$2;
  local -i tmp=0;
  for((i=0;i<=sz;i++)){
    ((a&(1<<i)))&&{ 
     ((tmp|=(1<<(sz-i)))); 
    }
  }
  echo $tmp;
}
#
function vMirror_bitmap(){
  local -i score=0;
  local -i sizeE=$((size-1));
  for((i=0;i<size;i++)){
    score=${scratch[$i]};
    trial[$i]=$(rh "$score" $sizeE);
  }
}
#
function intncmp_bs(){
  local -i rtn=0;
  for((i=0;i<size;i++)){
    rtn=$(echo "${board[$i]}-${scratch[$i]}"+10);
    ((rtn!=10))&&{ break; }
  }
  echo "$rtn";
}
#
function intncmp_bt(){
  local -i rtn=0;
  for((i=0;i<size;i++)){
    rtn=$(echo "${board[$i]}-${trial[$i]}"+10);
    ((rtn!=10))&&{ break; }
  }
  echo "$rtn";
}
function symmetryOps_bm(){
  local -i nEquiv=0;
  #回転・反転・対称チェックのためにboard配列をコピー
  for((i=0;i<size;i++)){ 
    trial[$i]=${board[$i]};
  }
  rotate_bitmap_ts; 
  #    //時計回りに90度回転
  k=$(intncmp_bs);
  ((k>10))&&{ 
   return; 
  }
  ((k==10))&&{ 
    nEquiv=2;
  }||{
    rotate_bitmap_st;
    #  //時計回りに180度回転
    k=$(intncmp_bt);
    ((k>10))&&{ 
     return; 
    }
    ((k==10))&&{ 
      nEquiv=4;
    }||{
      rotate_bitmap_ts;
      #//時計回りに270度回転
      k=$(intncmp_bs);
      ((k>10))&&{ 
        return;
      }
      nEquiv=8;
    }
  }
  #// 回転・反転・対称チェックのためにboard配列をコピー
  for((i=0;i<size;i++)){ 
    scratch[$i]=${board[$i]};
  }
  vMirror_bitmap;
  #//垂直反転
  k=$(intncmp_bt);
  ((k>10))&&{ 
   return; 
  }
  ((nEquiv>2))&&{
  #               //-90度回転 対角鏡と同等       
    rotate_bitmap_ts;
    k=$(intncmp_bs);
    ((k>10))&&{
      return;
    }
    ((nEquiv>4))&&{
    #             //-180度回転 水平鏡像と同等
      rotate_bitmap_st;
      k=$(intncmp_bt);
      ((k>10))&&{ 
        return;
      } 
      #      //-270度回転 反対角鏡と同等
      rotate_bitmap_ts;
      k=$(intncmp_bs);
      ((k>10))&&{ 
        return;
      }
    }
  }
  ((nEquiv==2))&&{
    ((COUNT2++));
  }
  ((nEquiv==4))&&{
    ((COUNT4++));
  }
  ((nEquiv==8))&&{
    ((COUNT8++));
  }
}
function backTrack2(){
	local -i bit;
  local -i min="$1";
	local -i left="$2";
	local -i down="$3";
	local -i right="$4";
	local -i bitmap=0;
  bitmap=$((MASK&~(left|down|right)));
	((min==size&&!bitmap))&&{
	  board[$min]=bitmap;
    symmetryOps_bm;
	}||{
    #枝刈りを追加
		((min<BOUND1))&&{  #上部サイド枝刈り
			((bitmap|=SIDEMASK));
			((bitmap^=SIDEMASK));
		} 
    #枝刈りを追加
		((min==BOUND2))&&{ #下部サイド枝刈り
				((!(down&SIDEMASK)))&& return ;
				(((down&SIDEMASK)!=SIDEMASK))&&((bitmap&=SIDEMASK));
		}
		while((bitmap));do
      #インラインによる最適化を追加
			((bitmap^=board[min]=bit=-bitmap&bitmap)); 
			backTrack2 "$((min+1))" "$(((left|bit)<<1))" "$((down|bit))" "$(((right|bit)>>1))"  ;
		done
	}
}
function backTrack1(){
	local -i bit;
  local -i min="$1";
	local -i left="$2";
	local -i down="$3";
	local -i right="$4";
	local -i bitmap=0;
  bitmap=$((MASK&~(left|down|right)));
	((min==size&&!bitmap))&&{
	  board[$min]=bitmap;
    #枝刈りを追加
    #symmetryOps_bm;
    ((COUNT8++));
	}||{
     #枝刈りを追加
		 ((min<BOUND1))&&{
			 	((bitmap|=2));
			 	((bitmap^=2));
		 }
		while((bitmap));do
      #インラインによる最適化を追加
			((bitmap^=board[min]=bit=(-bitmap&bitmap))) ;
			backTrack1 "$((min+1))" "$(((left|bit)<<1))" "$((down|bit))" "$(((right|bit)>>1))"  ;
		done
	}
}
#
function N-Queen11_rec(){
 	local -i min="$1";
	((TOPBIT=1<<(size-1)));
	board[0]=1;
	for((BOUND1=2;BOUND1<size-1;BOUND1++)){
	  bit=$((1<<BOUND1));
	  board[1]=$bit;
    backTrack1 "2" "$(((2|bit)<<1))" "$(((1|bit)))" "$((bit>>1))";
	}

  LASTMASK=$((TOPBIT|1));
	SIDEMASK=$LASTMASK;
	ENDBIT=$((TOPBIT>>1));
	BOUND2=$((size-2));
	for ((BOUND1=1;BOUND1<BOUND2;BOUND1++)){
	  bit=$((1<<BOUND1));
	  board[0]=$bit;
    backTrack2 "1" "$((bit<<1))" "$((bit))" "$((bit>>1))";
	  ((LASTMASK|=LASTMASK>>1|LASTMASK<<1));
	  ((ENDBIT>>=1));
	  ((BOUND2--));
	}
}
#
function N-Queen11(){
  local -i max=10;
	local -i min=2;
	local startTime=;
	local endTime= ;
	local hh=mm=ss=0; 		# いっぺんにに初期化することもできます
  echo " N:        Total       Unique        hh:mm:ss" ;
  for ((size=min;size<=max;size++)) {
    TOTAL=0;
		UNIQUE=0;
    COUNT2=COUNT4=COUNT8=0;
    for((j=0;j<$size;j++)){
     board[$j]=$j; 
    }
		MASK=$(((1<<size)-1));
    startTime=$(date +%s);# 計測開始時間
    N-Queen11_rec 0;
    endTime=$(date +%s); 	# 計測終了時間
    ss=$((endTime-startTime));# hh:mm:ss 形式に変換
    hh=$((ss/3600));
    ss=$((ss%3600));
    mm=$((ss/60));
    ss=$((ss%60));
    TOTAL=$(getTotal);
    UNIQUE=$(getUnique);
    printf "%2d:%13d%13d%10d:%.2d:%.2d\n" $size $TOTAL $UNIQUE $hh $mm $ss ;
  } 
}
#
  echo "<>１１．BT＋Bit＋対称解除Bit＋クイーンの位置による振り分け(BOUND1+BOUND2)＋枝刈り N-Queen11()";
  N-Queen11;
#

<>１１．BT＋Bit＋対称解除Bit＋クイーンの位置による振り分け(BOUND1+BOUND2)＋枝刈り N-Queen11()
 N:        Total       Unique        hh:mm:ss
 2:            0            0         0:00:00
 3:            0            0         0:00:00
 4:            2            1         0:00:00
 5:           10            2         0:00:00
 6:            4            1         0:00:00
 7:           40            6         0:00:00
 8:           92           12         0:00:00
 9:          352           46         0:00:02
10:          724           92         0:00:02


# Bash12

In [ ]:
%%bash

# グローバル変数
typeset -i TOTAL=0;
typeset -i UNIQUE=0;
typeset -i COUNT2=COUNT4=COUNT8=0;
typeset -i size=;
typeset -i sizeE=;
typeset -i MASK=SIDEMASK=LASTMASK=0;
typeset -i bit=TOPBIT=ENDBIT=0;
typeset -i BOUND1=BOUND2=0;
typeset -a board;
#
function symmetryOps(){
	((board[BOUND2]==1))&&{
		for((ptn=2,own=1;own<=sizeE;own++,ptn<<=1)){
			for((bit=1,you=sizeE;(board[you]!=ptn)&&(board[own]>=bit);you--)){
				((bit<<=1));
			}
			((board[own]>bit))&& return ;
			((board[own]<bit))&& break ;
		}
		#90度回転して同型なら180度回転も270度回転も同型である
		((own>sizeE))&&{ 
			((COUNT2++));
			return;
		}
	}
	#180度回転
	((board[sizeE]==ENDBIT))&&{ 
		for ((you=sizeE-1,own=1;own<=sizeE;own++,you--)){
			for ((bit=1,ptn=TOPBIT;(ptn!=board[you])&&(board[own]>=bit);ptn>>=1)){
					((bit<<=1)) ;
			}
			((board[own]>bit))&& return ;
			((board[own]<bit))&& break ;
		}
		#90度回転が同型でなくても180度回転が同型であることもある
		((own>sizeE))&&{ 
			((COUNT4++));
			return;
		}
	}
	#270度回転
	((board[BOUND1]==TOPBIT))&&{ 
		for((ptn=TOPBIT>>1,own=1;own<=sizeE;own++,ptn>>=1)){
			for((bit=1,you=0;(board[you]!=ptn)&&(board[own]>=bit);you++)){
					((bit<<=1)) ;
			}
			((board[own]>bit))&& return ;
			((board[own]<bit))&& break ;
		}
	}
	((COUNT8++));
}
#
# 最上段行のクイーンが角以外にある場合の探索 */
function Backtrack2(){
	local min=$1;
	local left=$2;
	local down=$3;
	local right=$4; 
	local bitmap=$((MASK&~(left|down|right)));
	((min==sizeE))&&{ 
		((bitmap))&&{
			((!(bitmap&LASTMASK)))&&{
					board[min]=$bitmap;
					symmetryOps ;
			}
		}
	}||{
    #枝刈り
		((min<BOUND1))&&{  #上部サイド枝刈り
			((bitmap|=SIDEMASK));
			((bitmap^=SIDEMASK));
		} 
    #枝刈り
		((min==BOUND2))&&{ #下部サイド枝刈り
				((!(down&SIDEMASK)))&& return ;
				(((down&SIDEMASK)!=SIDEMASK))&&((bitmap&=SIDEMASK));
		}
		while((bitmap));do
			((bitmap^=board[min]=bit=-bitmap&bitmap)); 
			Backtrack2 $((min+1)) $(((left|bit)<<1)) $(((down|bit)))  $(((right|bit)>>1)) ;
		done
	}
}
#
# 最上段行のクイーンが角にある場合の探索
function Backtrack1(){
	local min=$1;
	local left=$2;
	local down=$3;
	local right=$4; 
	local bitmap=$((MASK&~(left|down|right)));
	((min==sizeE))&&{
		 ((bitmap))&&{
			 	board[min]=$bm;
        #枝刈りによりsymmetryOpsは不要
				#symmetryOps ;
				((COUNT8++)) ;
		 }
	}||{
     #枝刈り
		 ((min<BOUND1))&&{
			 	((bitmap|=2));
			 	((bitmap^=2));
		 }
		 while((bitmap));do
			((bitmap^=board[min]=bit=(-bitmap&bitmap))) ;
			Backtrack1 $((min+1)) $(((left|bit)<<1))  $((down|bit)) $(((right|bit)>>1)) ;
		 done
	}
}
#
function func_BOUND1(){
	(($1<sizeE))&&{
		((board[1]=bit=1<<BOUND1));
		Backtrack1 2 $(((2|bit)<<1)) $((1|bit)) $((bit>>1));
	}
}
#
function func_BOUND2(){
	(($1<$2))&&{
		((board[0]=bit=1<<BOUND1));
		Backtrack2 1 $((bit<<1)) $bit $((bit>>1)) ;
	}
}
#
function N-QueenLogic_Q12(){
	board[0]=1;
	((sizeE=(size-1))); 	
	((MASK=(1<<size)-1));
	((TOPBIT=1<<sizeE));
	BOUND1=2;
	while((BOUND1>1&&BOUND1<sizeE));do
		func_BOUND1 "$BOUND1";
		((BOUND1++));
	done
	((SIDEMASK=LASTMASK=(TOPBIT|1)));
	((ENDBIT=TOPBIT>>1));
	BOUND1=1;
	((BOUND2=size-2));
	while((BOUND1>0&&BOUND2<sizeE&&BOUND1<BOUND2));do
		func_BOUND2 "$BOUND1" "$BOUND2";
		((BOUND1++,BOUND2--));
		((ENDBIT>>=1));
		((LASTMASK|=LASTMASK>>1|LASTMASK<<1)) ;
	done
	((UNIQUE=COUNT8+COUNT4+COUNT2)) ;
	((TOTAL=COUNT8*8+COUNT4*4+COUNT2*2));
}
#
function N-Queen12(){
  local -i max=10;
	local -i min=2;
	local startTime=0;
	local endTime=0;
	local hh=mm=ss=0; 		# いっぺんにに初期化することもできます
  echo " N:        Total       Unique        hh:mm:ss" ;
  for ((size=min;size<=max;size++));do
		COUNT2=COUNT4=COUNT8=0;
    startTime=$(date +%s);# 計測開始時間
		N-QueenLogic_Q12 ;
    endTime=$(date +%s); 	# 計測終了時間
    ss=$((endTime-startTime));# hh:mm:ss 形式に変換
    hh=$((ss/3600));
    ss=$((ss%3600));
    mm=$((ss/60));
    ss=$((ss%60));
    printf "%2d:%13d%13d%10d:%.2d:%.2d\n" $size $TOTAL $UNIQUE $hh $mm $ss ;
  done
}
#
  echo "<>１２．BT＋Bit＋対称解除Bit＋クイーンの位置による振り分け(BOUND1+BOUND2)＋枝刈り＋最適化 N-Queen12()";
  N-Queen12;
#

<>１２．BT＋Bit＋対称解除Bit＋クイーンの位置による振り分け(BOUND1+BOUND2)＋枝刈り＋最適化 N-Queen12()
 N:        Total       Unique        hh:mm:ss
 2:            0            0         0:00:00
 3:            0            0         0:00:00
 4:            2            1         0:00:00
 5:           10            2         0:00:00
 6:            4            1         0:00:00
 7:           40            6         0:00:00
 8:           92           12         0:00:00
 9:          352           46         0:00:00
10:          724           92         0:00:01
